# Q-learning, SARSA and Temporal Difference methods
Q-learning defines an algorithm to approximate $q_*$, the *optimal action-value function*. We can write it independently of the policy being followed, and it is guaranteed to converge to $q_*$ as long as all state-action pairs are visited infinitely often and the policy converges in the limit to the greedy policy.

\begin{equation}
Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \max_a Q(S_{t+1}, a) - Q(S_t, A_t) \right].
\end{equation}

In procedural form, the algorithm is:

\begin{equation}
\begin{split}
& \text{Initialize } Q(s, a), \text{ for all } s \in \mathcal{S}^+, a \in \mathcal{A}(s), \text{ arbitrarily except that } Q(\texttt{terminal}, \cdot) = 0 \\
& \text{Repeat (for each episode):} \\
& \quad \text{Initialize } S \\
& \quad \text{Repeat (for each step of episode):} \\
& \quad \quad \text{Choose } A \text{ from } S \text{ using policy derived from } Q \text{ (e.g., } \epsilon\text{-greedy)} \\
& \quad \quad \text{Take action } A, \text{ observe } R, S' \\
& \quad \quad Q(S, A) \leftarrow Q(S, A) + \alpha \left[ R + \gamma \max_a Q(S', a) - Q(S, A) \right] \\
& \quad \quad S \leftarrow S' \\
& \quad \text{until } S \text{ is terminal}
\end{split}
\end{equation}

